In [1]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

api_key = "97OGHWS77V9ZH0G7"
ts = TimeSeries(key=api_key, output_format="pandas")

# Get daily historical data for Tesla
tesla, tesla_meta = ts.get_daily(symbol="TSLA", outputsize="full")
tesla

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2025-03-04,270.930,284.3500,261.8401,272.04,125220232.0
2025-03-03,300.340,303.9400,277.3000,284.65,115551414.0
2025-02-28,279.500,293.8800,273.6000,292.98,115696968.0
2025-02-27,291.160,297.2300,280.8800,281.95,101748197.0
2025-02-26,303.715,309.0000,288.0400,290.80,100118276.0
...,...,...,...,...,...
2010-07-06,20.000,20.0000,15.8300,16.11,6866900.0
2010-07-02,23.000,23.1000,18.7100,19.20,5139800.0
2010-07-01,25.000,25.9200,20.2700,21.96,8218800.0


In [2]:
# Load 4 datasets (3 will merge to make ESG dataset, 1 commodities metal dataset)
esg = pd.read_csv("data/ESG World Bank Data/esg_data.csv")
environ = pd.read_csv("data/World Bank Environmental Data/environment.csv", skiprows=3)
metals = pd.read_csv("data/Commodities IMF Data/commodities_IMF.csv")
carbon = pd.read_csv("data/ESG World Bank Data/global_carbon_prices.csv")

carbon.head()

,Category,Shanghai pilot ETS,Norway carbon tax,Sweden carbon tax,BC carbon tax,Tokyo CaT,California CaT,Japan carbon tax,UK Carbon Price Support,France carbon tax,Korea ETS,China national ETS,Canada federal fuel charge,Netherlands carbon tax,Germany ETS,UK ETS,Australia CPM,Ontario CaT
0,1991,0.0,86.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1992,0.0,118.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,1993,0.0,101.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,1994,0.0,99.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,1995,0.0,115.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [3]:
# Reshape the ESG data into a long format
esg = esg.drop(columns='Series Code')
esg = esg.rename(columns={'Series Name': 'Indicator Name'})
esg_long = esg.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name'],
    var_name='Year',
    value_name='Value'
)
# Clean the 'Year' column to extract the numeric year
esg_long['Year'] = esg_long['Year'].str.extract(r'(\d{4})')

# Replace '..' with NaN to ensure missing values are correctly identified
esg_long['Value'] = pd.to_numeric(esg_long['Value'], errors='coerce')
esg_long.replace('..', pd.NA, inplace=True)
esg_long = esg_long.dropna(subset=['Value'])

# Select only the relevant countries (US, UK, China, EU countries: Germany, France, Netherlands, Sweden, etc.)
relevant_countries = ['United States', 'United Kingdom', 'China', 'Germany', 'France', 'Norway', 'Canada', 'Japan', 'South Korea', 'Netherlands', 'Sweden', 'Australia']

# Filter the ESG data
esg_df = esg_long[esg_long['Country Name'].isin(relevant_countries)]

# Environment dataset processing
environ = environ.drop(columns='Indicator Code')
environ_long = environ.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name'],
    var_name='Year',
    value_name='Value'
)
# Replace '..' with NaN to ensure missing values are correctly identified
environ_long.replace('..', pd.NA, inplace=True)
environ_long = environ_long.dropna(subset=['Value'])

# Filter the Environmental data for the relevant countries
environ_df = environ_long[environ_long['Country Name'].isin(relevant_countries)]

# Print a summary of the filtered datasets
esg_df.head()
environ_df.head()

,Country Name,Country Code,Indicator Name,Year,Value
1973,Australia,AUS,Total fisheries production (metric tons),1960,6.134500e+04
1974,Australia,AUS,Capture fisheries production (metric tons),1960,5.564500e+04
1975,Australia,AUS,Aquaculture production (metric tons),1960,5.700000e+03
2035,Australia,AUS,"Electricity production from renewable sources,...",1960,1.412653e+00
2036,Australia,AUS,"Electricity production from renewable sources,...",1960,3.030000e+08


In [4]:
# For esg_df
esg_indicators = ['Renewable energy consumption (% of total final energy consumption)',
                  'Electricity production from coal sources (% of total)', 
                  'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)',
                  'Regulatory Quality: Estimate',
                  'Research and development expenditure (% of GDP)', 
                  'Gini index',
                  'Patent applications, residents'] 
# For environ_df
environ_indicators = ['Nitrous oxide (N2O) emissions from Transport (Energy) (Mt CO2e)'] 

# EXCLUDED FOR NOW

# environ_indicators = 'Carbon dioxide (CO2) emissions from Transport (Energy) (Mt CO2e)',
                        # 'Methane (CH4) emissions from Transport (Energy) (Mt CO2e)'
                        
# esg_indicators = 'Political Stability and Absence of Violence/Terrorism: Estimate',
                   # 'Renewable electricity output (% of total electricity output)', 
                   # 'Methane emissions (metric tons of CO2 equivalent per capita)', 
                  #'Net migration',
                  # 'Renewable electricity output (% of total electricity output)', 
                  #'Population density (people per sq. km of land area)', 'GDP growth (annual %)', 


In [5]:
# Filter the ESG data for relevant indicators
esg_df = esg_df[esg_df['Indicator Name'].isin(esg_indicators)]

# Filter the Environmental data for relevant indicators
environ_df = environ_df[environ_df['Indicator Name'].isin(environ_indicators)]

# Combine the ESG and Environmental datasets
combined_df = pd.concat([esg_df, environ_df], ignore_index=True)

# Sort the data for better readability
esg_df = combined_df.sort_values(by=['Country Name', 'Year', 'Indicator Name'])
esg_df

,Country Name,Country Code,Indicator Name,Year,Value
2243,Australia,AUS,Nitrous oxide (N2O) emissions from Transport (...,1970,0.405900
2254,Australia,AUS,Nitrous oxide (N2O) emissions from Transport (...,1971,0.403700
2265,Australia,AUS,Nitrous oxide (N2O) emissions from Transport (...,1972,0.420500
2276,Australia,AUS,Nitrous oxide (N2O) emissions from Transport (...,1973,0.488100
0,Australia,AUS,Electricity production from coal sources (% of...,1974,67.782998
...,...,...,...,...,...
2230,United States,USA,Regulatory Quality: Estimate,2021,1.440957
2231,United States,USA,Research and development expenditure (% of GDP),2021,3.457050
2825,United States,USA,Nitrous oxide (N2O) emissions from Transport (...,2022,21.367400
2242,United States,USA,Regulatory Quality: Estimate,2022,1.424440


In [6]:
# Drop unnecessary columns
metals = metals.drop(columns=["Unnamed: 428", "Country Name", "Country Code", "Unit Code", "Attribute"], errors="ignore")

# Remove duplicates
metals = metals.drop_duplicates().reset_index(drop=True)

# Reshape Data to Long Format
metals = metals.melt(id_vars=["Commodity Name", "Commodity Code", "Unit Name"], 
                      var_name="Date", value_name="Value")

# Convert Date Column to datetime format
metals["Date"] = pd.to_datetime(metals["Date"].str.replace("M", "-", regex=False), format="%Y-%m")

# Handle missing values
metals = metals.dropna(subset=["Value"])

# Convert Value column to numeric
metals["Value"] = pd.to_numeric(metals["Value"], errors="coerce")

# Reset index after transformations
metals = metals.reset_index(drop=True)
metals = metals[metals['Unit Name'] == 'US Dollars']
metals

,Commodity Name,Commodity Code,Unit Name,Date,Value
2,Nickel,PNICK,US Dollars,1990-01-01,7056.000000
5,Aluminum,PALUM,US Dollars,1990-01-01,1528.000000
10,Nickel,PNICK,US Dollars,1990-02-01,6977.000000
13,Aluminum,PALUM,US Dollars,1990-02-01,1454.000000
18,Nickel,PNICK,US Dollars,1990-03-01,9267.000000
...,...,...,...,...,...
3947,Aluminum,PALUM,US Dollars,2024-12-01,2540.882273
3948,Lithium,PLITH,US Dollars,2024-12-01,96533.880271
3956,Nickel,PNICK,US Dollars,2025-01-01,15374.060870
3959,Aluminum,PALUM,US Dollars,2025-01-01,2571.370435


In [7]:
# Convert Date column to datetime format
metals["Date"] = pd.to_datetime(metals["Date"])

# Pivot so each commodity is a column
metals_pivot = metals.pivot(index="Date", columns="Commodity Name", values="Value")

# Resample to daily frequency and forward-fill missing values
metals_daily = metals_pivot.resample("D").ffill()

# Filter only data from June 2010 onward
metals_daily = metals_daily.loc["2010-06-01":].reset_index()

metals_daily

Commodity Name,Date,Aluminum,Lithium,Nickel
0,2010-06-01,1929.409091,NaN,19411.295455
1,2010-06-02,1929.409091,NaN,19411.295455
2,2010-06-03,1929.409091,NaN,19411.295455
3,2010-06-04,1929.409091,NaN,19411.295455
4,2010-06-05,1929.409091,NaN,19411.295455
...,...,...,...,...
5324,2024-12-28,2540.882273,96533.880271,15438.656364
5325,2024-12-29,2540.882273,96533.880271,15438.656364
5326,2024-12-30,2540.882273,96533.880271,15438.656364
5327,2024-12-31,2540.882273,96533.880271,15438.656364


In [8]:
# Convert Year to datetime (set to January 1st of each year)
esg_df["Date"] = pd.to_datetime(esg_df["Year"].astype(str) + "-01-01")

# Pivot so each indicator is a column
esg_pivot = esg_df.pivot_table(index=["Date", "Country Name"], 
                               columns="Indicator Name", 
                               values="Value")

# Resample to daily frequency and forward-fill missing values
esg_daily = esg_pivot.groupby(level="Country Name").apply(lambda x: x.resample("D").ffill()).reset_index()

# Filter only data from June 2010 onward
esg_daily = esg_daily[esg_daily["Date"] >= "2010-06-01"]

esg_daily


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'MultiIndex'

In [14]:
'''
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# Normalize Data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Function to create sequences
def create_sequences(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback, 0])  # Predicting Tesla Close Price
    return np.array(X), np.array(y)

# Create sequences
lookback = 60
X, y = create_sequences(data_scaled, lookback)

# Ensure correct shape
X = X.reshape(X.shape[0], X.shape[1], data_scaled.shape[1])

# Train-Test Split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(lookback, X.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

# Compile Model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train Model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Predictions
y_pred = model.predict(X_test)

# Rescale Predictions
y_pred_rescaled = scaler.inverse_transform(
    np.concatenate((y_pred, np.zeros((y_pred.shape[0], data_scaled.shape[1] - 1))), axis=1)
)[:, 0]

y_test_rescaled = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], data_scaled.shape[1] - 1))), axis=1)
)[:, 0]

# Plot Results
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled, label='Actual Price')
plt.plot(y_pred_rescaled, label='Predicted Price')
plt.legend()
plt.title('Tesla Stock Price Prediction')
plt.show()'''

,Tesla_Close,Aluminum,Lithium,Nickel
date,,,,
2025-02-28,292.98,NaN,NaN,NaN
2025-02-27,281.95,NaN,NaN,NaN
2025-02-26,290.80,NaN,NaN,NaN
2025-02-25,302.80,NaN,NaN,NaN
2025-02-24,330.53,NaN,NaN,NaN
...,...,...,...,...
2010-07-06,16.11,NaN,NaN,NaN
2010-07-02,19.20,NaN,NaN,NaN
2010-07-01,21.96,1989.045455,NaN,19548.522727
